In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
gc.collect()
df = pd.read_feather('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.feather')
df.head()

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1.0,10321418.0,45,3.0,3891.965283,3.0,0.0,0.0,0.0,1.00,...,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1.0
1,1.0,10414574.0,15,2.0,10616.561549,2.0,0.0,0.0,0.0,1.00,...,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1.0
2,1.0,10134567.0,48,2.0,23527.655416,0.0,2.0,0.0,0.0,0.00,...,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1.0
3,1.0,10001003.0,48,9.0,17751.558260,7.0,2.0,0.0,0.0,0.75,...,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1.0
4,1.0,10267183.0,2,1.0,21701.307598,1.0,0.0,0.0,0.0,1.00,...,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0.0


In [17]:
df.columns

Index(['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt',
       'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
       'domestic_offline_amt_pct', 'domestic_online_amt_pct',
       'overseas_offline_amt_pct', 'overseas_online_amt_pct', 'card_1_txn_cnt',
       'card_2_txn_cnt', 'card_3_txn_cnt', 'card_4_txn_cnt', 'card_5_txn_cnt',
       'card_6_txn_cnt', 'card_7_txn_cnt', 'card_8_txn_cnt', 'card_9_txn_cnt',
       'card_10_txn_cnt', 'card_11_txn_cnt', 'card_12_txn_cnt',
       'card_13_txn_cnt', 'card_14_txn_cnt', 'card_other_txn_cnt',
       'card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
       'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct',
       'card_7_txn_amt_pct', 'card_8_txn_amt_pct', 'card_9_txn_amt_pct',
       'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct',
       'card_13_txn_amt_pct', 'card_14_txn_amt_pct', 'card_other_txn_amt_pct',
       'masts', 'educd', 'trdtp', 'naty

In [18]:
df['dt'].min(),df['dt'].max()

(1.0, 24.0)

# 嘗試建立模型 
# 訓練集是 X = dt[1~22] Y = dt[23] 
# 測試集是 X = dt[2~23] Y = dt[24]

# 由於過去消費排名 不一定等於未來消費排名 (例如 人如果結婚了 升遷了 換工作了 遍有錢了都會導致消費型態的改變)

# 因此除了考慮過去消費排名可能還要加入其他特徵

In [19]:
other_features = ['masts', #婚姻
'educd', #教育
'trdtp', #行業別
'naty', # 國籍
'poscd', # 職位別
'cuorg', #客戶來源
'slam', #正卡信用額度
'gender_code', #信別
'age', #年紀
'primary_card', #正附卡註記
] 

# 這裡的想法是上面特徵都會影響一個人的消費行為
# 例如正卡信用額度大的人 可能會在"奢侈品"上花很多錢 
# 或者年紀大的人可能會在"保健食品"上花很多錢
# 由於 上面這些特徵都會"隨時間改變" 因此只須"考慮離測試數據最接近的狀態"就好

In [22]:
df_sorted = df.loc[(df.dt>=20)&(df.dt<=24)] # 考慮離測試數據最接近的狀態
df_sorted = df_sorted.sort_values(by='dt')
print(df_sorted.shape)
chid_mapping = {}
for fname in tqdm(other_features):
    chid_mapping[fname] = dict(zip(df_sorted['chid'],df_sorted[fname])) # chid到"特徵"的映射
chid_mapping.keys()
df_sorted

 10%|█         | 1/10 [00:02<00:22,  2.45s/it]

(8002031, 53)


100%|██████████| 10/10 [00:23<00:00,  2.38s/it]


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
24973529,20.0,10294163.0,36,3.0,9627.458307,3.0,0.0,0.0,0.0,1.00,...,2.0,2.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1.0
26015531,20.0,10292037.0,15,4.0,16716.782699,4.0,0.0,0.0,0.0,1.00,...,1.0,3.0,15.0,1.0,99.0,30.0,74152.141012,0.0,3.0,1.0
26015530,20.0,10286187.0,10,1.0,7142.555210,0.0,1.0,0.0,0.0,0.00,...,1.0,3.0,8.0,1.0,4.0,30.0,52676.553882,1.0,3.0,1.0
26015529,20.0,10071616.0,2,4.0,72538.809556,4.0,0.0,0.0,0.0,1.00,...,2.0,3.0,15.0,1.0,99.0,30.0,79823.876682,1.0,3.0,1.0
26015528,20.0,10039366.0,10,1.0,6627.410370,0.0,1.0,0.0,0.0,0.00,...,1.0,4.0,21.0,1.0,1.0,30.0,102382.569589,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31919452,24.0,10113264.0,48,2.0,8019.906666,2.0,0.0,0.0,0.0,1.00,...,2.0,4.0,5.0,1.0,99.0,30.0,60923.663994,1.0,4.0,1.0
31919451,24.0,10483631.0,other,10.0,15958.338184,3.0,0.0,0.0,7.0,0.56,...,2.0,3.0,15.0,1.0,99.0,30.0,NaN,1.0,3.0,0.0
31919450,24.0,10133159.0,37,3.0,11820.489438,3.0,0.0,0.0,0.0,1.00,...,1.0,4.0,2.0,1.0,2.0,30.0,62730.806297,1.0,4.0,1.0
31919424,24.0,10193187.0,31,8.0,10614.092737,1.0,7.0,0.0,0.0,0.16,...,2.0,3.0,2.0,1.0,1.0,30.0,79291.240239,1.0,3.0,1.0


# create_x_y 函數

In [23]:
def create_x_y(df,split_dt,other_features,chid_mapping,mode='train'):
    def add_topk(df,col):
        var = np.argsort(np.array(df[col].values.tolist()))
        df[f'top{1}'] = var[:,-1]
        df[f'top{2}'] = var[:,-2]
        df[f'top{3}'] = var[:,-3]
        return df
    官方指認欄位 = [2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48]
    官方指認欄位 = ['shop_tag_' + str(i) for i in 官方指認欄位]
    origin_df = df
    def create_features(df,split_dt,look_back=3):
        df = df.loc[(df.dt>=split_dt-look_back)&(df.dt<=split_dt)] #根據dt做row的篩選
        df = df.pivot_table(index=['chid'], columns='shop_tag', values='txn_amt',aggfunc=np.sum).fillna(0.0) # 計算這個chid(index)在各shop_tag(columns)的txn_amt(values)
        df.columns = [ 'shop_tag_' + str(i) for i in df.columns] # 欄位加上前綴
        df = df.loc[:,官方指認欄位] # 只取官方指認欄位
        df['chid'] = df.index
        for fname in tqdm(other_features):
            df[fname] = df['chid'].apply(lambda x:chid_mapping[fname][x]) # chid到特徵的映射 
        return df
    x = create_features(df.loc[df.dt < split_dt,:],split_dt) # time == [t-1 ,t-2 ... ,t-n]
    y = create_features(df.loc[df.dt == split_dt,:],split_dt) # time == t
    common_index = set(x.index) & set(y.index) # 取共同index
    x,y = x.loc[common_index],y.loc[common_index]
    y = add_topk(y,col=y.columns[y.columns.str.contains('shop_tag')]) # add topk columns
    y = y.loc[:,y.columns[y.columns.str.contains('top')]] # columns axis select topk columns
    官方指認欄位 = [2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48]
    mapping = dict(zip([*range(len(官方指認欄位))],官方指認欄位))
    for col in y.columns:
        y[col] = y[col].apply(lambda x:mapping[x]) # 轉換
    print('x shape',x.shape,'y shape',y.shape)
    return x,y

In [24]:
X_train, y_train = create_x_y(df,split_dt=23,other_features=other_features,chid_mapping=chid_mapping,mode='train')
X_test, y_test = create_x_y(df,split_dt=24,other_features=other_features,chid_mapping=chid_mapping,mode='test')

100%|██████████| 10/10 [00:02<00:00,  3.55it/s]


x shape (419727, 27) y shape (419727, 3)


100%|██████████| 10/10 [00:02<00:00,  3.66it/s]


x shape (420732, 27) y shape (420732, 3)


In [25]:
X_train

,shop_tag_2,shop_tag_6,shop_tag_10,shop_tag_12,shop_tag_13,shop_tag_15,shop_tag_18,shop_tag_19,shop_tag_21,shop_tag_22,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
chid,,,,,,,,,,,,,,,,,,,,,
10485760.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,1.0,4.0,8.0,1.0,1.0,12.0,102382.569589,0.0,6.0,1.0
10485761.0,42417.770262,0.000000,76522.964200,0.0,0.000000,27966.344293,0.000000,0.000000,0.000000,0.0,...,2.0,3.0,2.0,1.0,4.0,30.0,102382.569589,1.0,3.0,1.0
10485762.0,15958.303518,0.000000,18527.238020,0.0,24371.585109,8677.196788,3480.859743,0.000000,0.000000,0.0,...,2.0,3.0,9.0,1.0,99.0,30.0,44545.625512,0.0,3.0,1.0
10485763.0,45026.598289,0.000000,0.000000,0.0,0.000000,20236.239795,3324.768457,0.000000,0.000000,0.0,...,1.0,2.0,11.0,1.0,2.0,30.0,99935.984396,1.0,4.0,1.0
10485764.0,0.000000,0.000000,0.000000,0.0,0.000000,10965.958583,0.000000,9277.143937,0.000000,0.0,...,1.0,4.0,12.0,1.0,99.0,30.0,84815.219772,0.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10485754.0,0.000000,0.000000,45937.403176,0.0,0.000000,5540.285984,10801.793485,0.000000,0.000000,0.0,...,2.0,3.0,12.0,1.0,2.0,30.0,87397.101503,0.0,3.0,1.0
10485755.0,34509.343013,0.000000,25676.391759,0.0,9958.409338,9373.392018,0.000000,0.000000,0.000000,0.0,...,2.0,3.0,15.0,1.0,2.0,30.0,38072.451849,0.0,2.0,1.0
10485756.0,0.000000,25004.783761,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,2.0,6.0,5.0,1.0,2.0,30.0,86109.761716,0.0,3.0,1.0


In [26]:
y_train

,top1,top2,top3
chid,,,
10485760.0,48,39,37
10485761.0,36,10,13
10485762.0,10,2,36
10485763.0,26,2,15
10485764.0,37,36,48
...,...,...,...
10485754.0,10,37,26
10485755.0,2,36,12
10485756.0,6,48,39


In [27]:
X_test

,shop_tag_2,shop_tag_6,shop_tag_10,shop_tag_12,shop_tag_13,shop_tag_15,shop_tag_18,shop_tag_19,shop_tag_21,shop_tag_22,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
chid,,,,,,,,,,,,,,,,,,,,,
10485760.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,1.0,4.0,8.0,1.0,1.0,12.0,102382.569589,0.0,6.0,1.0
10485761.0,12289.330042,0.000000,62643.191930,0.000000,2688.055269,22366.738610,0.000000,0.000000,0.000000,0.0,...,2.0,3.0,2.0,1.0,4.0,30.0,102382.569589,1.0,3.0,1.0
10485762.0,27038.812431,0.000000,29267.566728,0.000000,24586.212696,16555.646052,0.000000,0.000000,0.000000,0.0,...,2.0,3.0,9.0,1.0,99.0,30.0,44545.625512,0.0,3.0,1.0
10485763.0,52224.193152,0.000000,0.000000,0.000000,0.000000,22907.242157,7225.739353,0.000000,0.000000,0.0,...,1.0,2.0,11.0,1.0,2.0,30.0,99935.984396,1.0,4.0,1.0
10485764.0,0.000000,0.000000,0.000000,0.000000,0.000000,9331.575338,0.000000,9277.143937,0.000000,0.0,...,1.0,4.0,12.0,1.0,99.0,30.0,84815.219772,0.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10485755.0,25003.263736,0.000000,31420.977151,6303.494132,9218.267691,2939.762251,0.000000,0.000000,0.000000,0.0,...,2.0,3.0,15.0,1.0,2.0,30.0,38072.451849,0.0,2.0,1.0
10485756.0,0.000000,25004.783761,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,2.0,6.0,5.0,1.0,2.0,30.0,86109.761716,0.0,3.0,1.0
10485757.0,36788.182809,0.000000,20688.378027,0.000000,13799.666117,47992.206162,12225.410860,6148.896046,25075.366933,0.0,...,2.0,3.0,22.0,1.0,1.0,30.0,74152.141012,0.0,3.0,1.0


In [28]:
y_test

,top1,top2,top3
chid,,,
10485760.0,48,39,37
10485761.0,15,36,10
10485762.0,19,13,36
10485763.0,15,39,48
10485764.0,19,48,39
...,...,...,...
10485755.0,10,2,36
10485756.0,6,15,48
10485757.0,15,2,18


In [29]:
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(419727, 27) (420732, 27)
(419727, 3) (420732, 3)


In [30]:
X_train.reset_index(drop=True).to_feather('../data/X_train.feather')
y_train.reset_index(drop=True).to_feather('../data/y_train.feather')
X_test.reset_index(drop=True).to_feather('../data/X_test.feather')
y_test.reset_index(drop=True).to_feather('../data/y_test.feather')